## Эмбеддинги (Embeddings)

В предыдущем разделе мы работали с многомерными векторами BoW с длиной `vocab_size`, при этом мы явно преобразовывали низкоразмерные векторы позиционного представления в разреженный one-hot encoding. С одной стороны, это приводит к существенному расходу памяти. С другой, разные слова представляются разными векторами, расстояние (в терминах вектоного пространства) между которыми для всех пар слов одинаково. Поэтому близость между векторами никак не отражает близость их смысла.

В этом разделе мы продолжим изучение набора данных **News AG**.

In [6]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()


### Что такое эмбеддинги

Идея **эмбеддинга** состоит в том, чтобы представить слова с использованием низкоразмерных плотных векторов, которые в идеале отражают семантическое значение слова. Позже мы обсудим, как создавать семантически-осмысленные эмбеддинги, но пока давайте просто подумаем об этом как о способе уменьшить размерность вектора слова. 

Эмбеддинги реализуются в виде специального слоя `Embedding`, который принимает на вход номер слова в словаре и создает выходной вектор заданного `embedding_size`. В некотором смысле, он очень похож на слой `Dense`, но вместо того, чтобы принимать в качестве входных данных one hot encoding, он принимает число.

Используя такой слой в качестве первого слоя в нашей сети, мы можем переключиться с BoW на модель **embedding bag**, где мы сначала преобразуем каждое слово в нашем тексте в соответствующий embedding, а затем вычисляем некоторую совокупную функцию для всех таких представлений слова - это может быть сумма или среднее:  

![Image showing an embedding classifier for five sequence words.](./images/embedding-classifier-example.png)

Наша нейронная сеть классификатора состоит из следующих слоев:

* Слой `TextVectorization`, который принимает строку в качестве входных данных и выдает вектор номеров всех токенов в словаре. Мы укажем некоторый разумный размер словаря `vocab_size` и проигнорируем менее часто используемые слова. Размерность входных данных - 1, размерность выходных данных - длина последовательности $n$, так как в результате мы получим $n$ токенов, каждый из которых содержит числа от 0 до `vocab_size`.
* Слой `Embedding`, который принимает $n$ чисел и сводит каждое число к плотному вектору заданной длины (100 в нашем примере). Таким образом, входной тензор размерности $n$ будет преобразован в тензор $n\times 100$. 
* Слой агрегирования, который вычисляет среднее значение этого тензора вдоль первой оси, т.е. вычисляет среднее значение всех входных тензоров $n$, соответствующих разным словам. Для реализации этого слоя мы будем использовать слой 'Lambda', и передавать в него функцию для вычисления среднего значения. Выход будет иметь размерность 100, и это будет числовое представление всей входной последовательности.
* Конечный линейный классификатор `Dense`.

In [7]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, None)              0         
_________________________________________________________________
embedding (Embedding)        (None, None, 100)         3000000   
_________________________________________________________________
lambda (Lambda)              (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 4)                 404       
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


В распечатке `summary` в столбце **output shape** первое тензорное измерение `None` соответствует размеру минибэтча, а второе - длине последовательности токенов. Все последовательности токенов в минибатче имеют разную длину. О том, как с этим бороться, мы поговорим в следующем разделе.

Теперь давайте обучим сеть:

In [8]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

In [ ]:
print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

> **Обратите внимание**, что мы строим векторизатор на основе подмножества данных. Это делается для того, чтобы ускорить процесс, и это может привести к ситуации, когда не все токены из нашего текста присутствуют в словаре. В этом случае эти токены будут проигнорированы, что может привести к немного более низкой точности. Однако в реальной жизни подмножество текста часто дает хорошую оценку словарного запаса.

### Работа с переменными размерами последовательностей

Давайте разберемся, как происходит обучение в мини-батчах в случае с последовательностями разной длины. В приведенном выше примере входной тензор имеет размерность 1, и мы используем минибатчи размером 128, так что фактический размер тензора составляет $128 \times 1$. Однако количество токенов в каждом предложении отличается. Если мы применим слой `TextVectorization` ко входной строке, количество возвращаемых токенов будет отличаться в зависимости от строки:

In [13]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


Однако, когда мы применяем векторизатор к нескольким последовательностям, он должен выдавать тензор прямоугольной формы, поэтому он заполняет неиспользуемые элементы токеном PAD (который в нашем случае равен нулю):

In [14]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

Кстати, можем посмотреть на сами эмбеддинги:

In [15]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 0.0762681 ,  0.0472152 ,  0.00431035, ..., -0.00250225,
         -0.01864387,  0.02116265],
        [ 0.19173232,  0.2793066 , -0.01789329, ...,  0.04434786,
          0.14992206, -0.00824973],
        [ 0.0309074 ,  0.02908396,  0.00316487, ..., -0.03633584,
         -0.03174757, -0.04419322],
        [ 0.0309074 ,  0.02908396,  0.00316487, ..., -0.03633584,
         -0.03174757, -0.04419322],
        [ 0.0309074 ,  0.02908396,  0.00316487, ..., -0.03633584,
         -0.03174757, -0.04419322],
        [ 0.0309074 ,  0.02908396,  0.00316487, ..., -0.03633584,
         -0.03174757, -0.04419322]],

       [[ 0.21704227,  0.30581048,  0.1079862 , ..., -0.10363591,
          0.03489591,  0.17299455],
        [ 0.08685672,  0.1604307 ,  0.14692512, ..., -0.09063095,
         -0.02848253,  0.08478612],
        [ 0.0762681 ,  0.0472152 ,  0.00431035, ..., -0.00250225,
         -0.01864387,  0.02116265],
        [-0.12317707, -0.08362484, -0.01695838, ..., -0.05036105,
         -0.05

> **Примечание**: Чтобы минимизировать padding, в некоторых случаях имеет смысл сортировать все последовательности в наборе данных в порядке увеличения длины (или, точнее, количества токенов). Это гарантирует, что каждая минибатч содержит последовательности минимально отличающейся длины.

## Семантические эмбеддинги: Word2Vec

В нашем предыдущем примере слой embedding научился сопоставлять слова с векторными представлениями, однако эти представления не имели семантического значения. Было бы неплохо обучить векторное представление таким образом, чтобы похожие слова или синонимы соответствовали векторам, близким друг к другу с точки зрения некоторого векторного расстояния (например, евклидова расстояния).

Для этого нам нужно предварительно обучить модель эмбеддинга на основе некоторой большой коллекции текстов, используя такую технику semi-supervised learning, как [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Он основан на двух основных архитектурах, которые используются для создания векторного представления слов:

 - **Непрерывный мешок слов** (Continuous Bag of Words, CBoW), где мы обучаем модель предсказывать слово из окружающего контекста. Рассматриваем n-грамму $(W_{-2},W_{-1},W_0,W_1,W_2)$, целью модели является прогнозирование $W_0$ по $(W_{-2},W_{-1},W_1,W_2)$.
 - **Continuous skip-gram** противоположен CBoW. Модель использует окружающее окно контекстных слов для прогнозирования текущего слова.

![Image showing both CBoW and Skip-Gram algorithms to convert words to vectors.](images/example-algorithms-for-converting-words-to-vectors.png)

Чтобы поэкспериментировать с предварительно обученной моделью Word2Vec, мы можем использовать библиотеку **gensim**. Ниже мы находим слова, наиболее похожие на `neural`

> **Примечание:** При первом запуске загрузка векторов слов может занять некоторое время!

In [2]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [3]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326499819755554
neural_circuits -> 0.7252851128578186
neuron -> 0.7174385786056519
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.669911801815033
neural_circuitry -> 0.6638562679290771
neurochemical -> 0.6555314660072327
neuronal_activity -> 0.6531826257705688


Для получения значения эмбеддинга, соответствующего какому-то слову, мы можем использовать индексацию `w2v`. Размерность эмбеддинга - 300 компонентов, но здесь мы для наглядности показываем только первые 20:

In [18]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Самое замечательное в семантических эмбеддингах заключается в том, что вы можете манипулировать семантикой на основе векторного представления. Например, мы можем попросить найти слово, векторное представление которого максимально приближено к словам *король* и *женщина*, и как можно дальше от слова *мужчина*:

In [19]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118193507194519)

Интересным свойством семантических эмбеддингов является то, что мы можем выполнять обычные векторные операции над эмбеддингами, и это будет отражаться в соответствующих операциях со **смыслом**. Приведенный выше пример можно выразить в терминах векторных операций: вычисляем вектор, соответствующий **KING-MAN+WOMAN** (операции '+' и '-' выполняются на векторных представлениях соответствующих слов), а затем находим в словаре ближайшее к этому вектору слово:

In [4]:
# получаем векторное представление целевого слова
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# ищем индекс ближайшего эмбеддинг-вектора 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# выводим само слово
w2v.index_to_key[min_idx]

'queen'

> **ПРИМЕЧАНИЕ**: Нам пришлось добавить небольшие коэффициенты к векторам *мужчина* и *женщина* - попробуйте удалить их и посмотреть, что произойдет.

Чтобы найти ближайший вектор, мы используем механизм numpy для вычисления вектора расстояний между целевым вектором и всеми векторами в словаре, а затем находим индекс минимального слова с помощью `argmin`.

Word2Vec кажется отличным способом векторного представления семантики слов, однако он имеет ряд недостатков:

* Модели CBoW и Skip-Gram являются **предиктивными эмбеддингами**, они учитывают только локальный контекст. Word2Vec не использует преимущества глобального контекста.
* Word2Vec не учитывает **морфологию**, т.е. тот факт, что значение может зависеть от разных частей слова, таких, как корень.  

Для преодоления второго ограничения была создана модель **FastText**. Ещё одна модель семантических эмбеддингов - это **GloVe**. Библиотека gensim также поддерживает эти эмбеддинги, и вы можете поэкспериментировать с ними, изменив приведенный выше код загрузки модели.

## Использование предобученных эмбеддингов в Keras

Мы можем изменить рассмотренный ранее пример классификации и предварительно заполнить матрицу в эмбеддинг-слое значениями из Word2Vec. Однако при этом возникнет одна проблема - словари предобученного word2vec и нашего текстового корпуса, скорее всего, не будут совпадать. Здесь мы рассмотрим два возможных варианта преодоления проблемы: использование словаря токенизатора и использование словаря из Word2Vec.

### Использование словаря токенизатора

При использовании словаря токенизатора некоторые слова из словаря будут иметь соответствующие вектора Word2Vec, а некоторые будут отсутствовать. Учитывая, что размер нашего словаря равен `vocab_size`, а длина вектора Word2Vec — `embed_size`, эмбеддинг-слой будет описываться матрицей весов размером `vocab_size`$\times$`embed_size`. Мы заполним эту матрицу векторами из Word2Vec, оставляя строки матрицы, для которых нет соответствующих Word2Vec векторов, пустыми (или же инициализируя случайными значениями):

In [21]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


Для слов, которых нет в словаре Word2Vec, мы можем либо оставить их нулевыми, либо сгенерировать случайный вектор.

Теперь мы можем определить эмбеддинг-слой с предварительно подготовленной матрицей весов:

In [22]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

Обучим нашу модель: 

In [23]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 13s 12ms/step - loss: 1.1048 - acc: 0.7937 - val_loss: 0.9113 - val_acc: 0.8182


> **Примечание**: Обратите внимание, что мы устанавливаем `trainable=False` при создании слоя `Embedding`, что означает, что мы не переобучаем Embedding-вектора Word2Vec. Это может привести к тому, что точность будет немного ниже, но это ускоряет тренировку. В дальнейшем можно поступить также, как и при Transfer Learning для изображений, и разморозить веса в какой-то момент.

### Использование словаря Word2Vec

Прежде всего, мы создадим слой `TextVectorization` с заданным словарем, взятым из Word2Vec:

In [9]:
vocab = w2v.index_to_key
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

Теперь мы можем инициализировать веса эмбеддинг-слоя непосредственно из весов Word2Vec:

In [11]:
model = keras.models.Sequential([
    vectorizer, 
    keras.layers.Embedding(len(vocab), embed_size, weights=[w2v.vectors], trainable=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 4326s 5s/step - loss: 1.3386 - acc: 0.4960 - val_loss: 1.3000 - val_acc: 0.5695


Одна из причин, по которой мы не видим более высокой точности, заключается в том, что некоторые слова из нашего датасета отсутствуют в предварительно обученном словаре Word2Vec, и, следовательно, они по существу игнорируются. Чтобы преодолеть это, мы можем обучить наши собственные Word2Vec эмбеддинги на основе нашего набора данных. 

## Контекстные эмбеддинги

Одним из ключевых ограничений традиционных предобученных эмбеддингов является то, что они не могут различать зависящие от контекста значения слов. Например, слово *play* имеет разное значение в этих двух предложениях:

- I went to a **play** at the theater.
- John wants to **play** with his friends.

Предварительно обученные эмбеддинги представляют оба значения слова одним и тем же вектором. Чтобы преодолеть это ограничение, нам нужно построить эмбеддинги на основе более богатой **языковой модели**, которая учитывает порядок слов, обучается на большом корпусе текста и *знает*, как слова могут быть собраны вместе в разных контекстах. Обсуждение контекстных эмбеддингов выходит за рамки этого курса, но мы вернемся к проблеме контекста, когда будем говорить о рекуррентных сетях.
